****
## **MODELO DE ELEMENTOS FINITOS QUE SIMULA EL PROCESO DE CALENTAMIENTO DE UNA PALANQUILLA A SU PASO POR UN HORNO DE INDUCCIÓN**
****

**Autor**: Sergio Bolívar Gómez.

**Fecha**: 10 de abril de 2023.

En primer lugar, cargamos las librerías que utilizaremos tanto para realizar la simulación como para representar gráficamente los resultados obtenidos.

In [1]:
import os # importamos la librería os para poder interactuar con el sistema operativo
import pandas as pd # importamos la librería pandas para manejar datos (uso de DataFrames)
import numpy as np # importamos la librería numpy para hacer cálculos numéricos
import matplotlib.pyplot as plt # importamos la librería pyplot para representar gráficamente los resultados
import time # importamos la librería time para formatear el tiempo transcurrido

from ansys.mapdl.core import launch_mapdl # importamos la función launch_mapdl del módulo ansys.mapdl.core para conectarnos a ANSYS APDL

Una vez importadas las librerías necesarias, definimos una serie de funciones que nos van a facilitar la lectura de los datos. Sacando estas funciones del propio código ganamos modularidad, de manera que si los formatos proporcionados por GSW son algo complejos, podemos modificar estas funciones para adecuarlas a su lectura.

In [2]:
def load_case_studies_data(filename):
    """
    Carga los datos sobre los casos de estudio desde un CSV con las siguiente columnas:
    tiempo, avance, IND_1, IND_2, IND_3, IND_4
    
    Parametros:
    ----------
    filename (str): nombre del CSV que contiene los datos sobre los casos de estudio
    
    Output:
    ------
    data (pandas.DataFrame): un pandas DataFrame que contiene los datos
    
    """
    data = pd.read_csv(filename, header=0, skipinitialspace=True)
    return data


def load_material_properties(filename):
    """
    Carga los datos de propiedades de material desde un CSV con las siguientes columnas:
    conductividad, calor_especifico, densidad, coef_conv_air_acero

    Parametros:
    ----------
    filename (str): nombre del archivo csv que contiene los datos de las propiedades del material

    Returns
    -------
    data (pandas.DataFrame): un pandas DataFrame que contiene los datos

    """
    data = pd.read_csv(filename, header=0, skipinitialspace=True)
    return data


def load_boundary_conditions(filename):
    """
    Carga los datos sobre las condiciones de contorno desde un CSV con las siguientes columnas:
    temperatura_inicial, temperatura_aire

    Parametros:
    ----------
    filename (str): nombre del archivo csv que contiene los datos de las condiciones de contorno

    Returns
    -------
    data (pandas.DataFrame): un pandas DataFrame que contiene los datos

    """
    data = pd.read_csv(filename, header=0, skipinitialspace=True)
    return data

In [3]:
mapdl = launch_mapdl(nproc=10) # lanzamos una instancia de MAPDL y la asignamos a la variable mapdl

Elaboramos una lista con todas los identificadores de todas las palanquillas...

In [4]:
directory = f"C:\\Users\\bolivars\\Desktop\\JAE\\CODIGO\\CODIGO\\inputs-APDL\\Parametros_casos_estudio"

casos_estudio = []

for filename in os.listdir(directory):
    if filename.startswith("caso_estudio_palanquilla_") and filename.endswith(".csv"):
        number = filename.split("_")[-1].split(".")[0]
        casos_estudio.append(number)

casos_estudio.sort()

In [5]:
print("El número de casos de estudio analizados son", len(casos_estudio))

El número de casos de estudio analizados son 159


Comienza la simulación...

In [9]:
start_time_simul = time.time()

total_advance = 0 # avance de la palanquilla (esta variable no está definida en el código original, REVISAR)

dir_path = f"C:\\Users\\bolivars\\Desktop\\JAE\\CODIGO\\CODIGO\\inputs-APDL" # path al directorio de trabajo base

os.chdir(dir_path + "\Caso_0")

mapdl.parsav("SCALAR", 'paramentros_caso') # escribe los parámetros en un fichero

mapdl.config("NRES", 30000) # aumentamos el número máximo de substeps

for case_id in casos_estudio:

    print(75*"=")
    print(13*" ","COMIENZA LA SIMULACIÓN - PALANQUILLA",case_id)
    print(75*"=")

    start_time_case_study = time.time()

    # Cambiamos el directorio de trabajo al del caso 0, que contiene el estado incial para el análisis
    os.chdir(dir_path + "\Caso_0")
    mapdl.cwd(dir_path + "\Caso_0")

    mapdl.resume("estado_inicial_2_analisis", "db") # recupera la base de datos desde el archivo proporcionado
    mapdl.parres("NEW", "paramentros_caso") # lee los parámetros desde el archivo que le pasamos 
    mapdl.parsav("SCALAR", "paramentros_caso") # escribe los parámetros en un fichero

    # Desactivamos los pop-ups (para poder generar varios casos en serie)
    mapdl.keyw("PR_SGVOF", 1)
    mapdl.run("/NERR,5,10000, ,0,1000")
    
    mapdl.prep7() # entramos en el preproceso

    # Activamos la numeración en pantalla de puntos, líneas...
    mapdl.pnum("KP",1)
    mapdl.pnum("LINE",1)
    mapdl.pnum("AREA",1)
    mapdl.pnum("VOLU",1)
    mapdl.pnum("NODE",0)

    # Variables de la simulación (en principio son fijas, pero quizá también puedan leerse desde algún fichero)
    num_steps = 200 # para leer los ficheros de texto en orden
    influence = 1.1 # influencia del inductor
    sep_inductors = 0.5 # separación entre los inductores
    constant_loss = 50000 # a ajustar (está puesta a ojo, REVISAR)
    time_init_step = 0 # cuando hay varios casos, para que el tiempo inicial no se inferior al último del cálculo anterior

    # Cambiamos el directorio de trabajo al lugar donde se almacenan los parámetros fijos del modelo
    os.chdir(dir_path + "\Parametros_fijos_modelo")
    mapdl.cwd(dir_path + "\Parametros_fijos_modelo")

    mat_prop = load_material_properties("Propiedades_material.txt") # leemos las propiedades del material

    # Propiedades del material
    conductivity = mat_prop.loc[0,"conductividad"] # conductividad en W/(m·K)
    specific_heat = mat_prop.loc[0,"calor_especifico"] # calor específico en J/(kg·K)
    density = mat_prop.loc[0,"densidad"] # densidad en kg/m^3 
    conv_coef = mat_prop.loc[0,"coef_conv_air_acero"] # coeficiente de convección acero-aire (W/(m^2·K))

    # Geometría de la placa
    e = 0.14   # alto (m)
    b = 0.14   # ancho (m)
    a = 19.5   # largo (m)

    # Tamaño de los elementos de la malla
    elem_mesh_size = 0.02

    #############################
    ##### DEFINIR ELEMENTOS #####
    #############################

    mapdl.et(1, "SOLID70") # elemento 3D (3-D thermal conduction capability)

    #################################################
    ##### DEFINIR PROPIEDADES DE LOS MATERIALES #####
    #################################################

    mapdl.mptemp()                           # borra tabla de temperaturas
    mapdl.mptemp(1,1.0000000E+03)            # creamos una nueva tabla de temperaturas con identificador 1 y temperatura inicial de 1000 K
    mapdl.mpdata("DENS", 1, 1, density)      # añadimos datos a la tabla de propiedades del material: densidad
    mapdl.mptemp()                           # borra tabla de temperaturas
    mapdl.mptemp(1,1.0000000E+03)            # creamos una nueva tabla de temperaturas con identificador 1 y temperatura inicial de 1000 K
    mapdl.mpdata("KXX", 1, 1, conductivity)  # añadimos datos a la tabla de propiedades del material: conductividad térmica
    mapdl.mptemp(1,1.0000000E+00)               
    mapdl.mpdata("C", 1, 1, specific_heat)   # añadimos datos a la tabla de propiedades del material: calor específico

    ###############################################
    ##### DEFINIR LA GEOMETRÍA DEL COMPONENTE #####
    ###############################################

    mapdl.block(0, -a, 0, b, 0, e) # volumen que representa la palanquilla
    mapdl.view(1, 1, 2, 3) # ver en perspectiva
    mapdl.angle(1)
    mapdl.replot("FAST")

    ##########################################
    ##### DEFINIR MALLADO DEL COMPONENTE #####
    ##########################################
    
    mapdl.esize(elem_mesh_size, 5) # establecemos el tamaño de los elementos (segundo parámetro?)
    mapdl.mshape(0, "3D")
    mapdl.mshkey(1)

    mapdl.cm('_Y', 'VOLU') # creamos componente
    mapdl.vsel('', '', '', 1)
    mapdl.cm('_Y1', 'VOLU') # creamos componente
    mapdl.chkmsh('VOLU')
    mapdl.cmsel('S', '_Y')

    mapdl.vmesh('_Y1') # hacemos el mesh (genera nodos y elementos)

    # Eliminamos los componentes creados previamente
    mapdl.cmdele('_Y')
    mapdl.cmdele('_Y1')

    #########################
    ##### DEFINIR CASOS #####
    #########################

    mapdl.cswpla(12,0,0,0) # creamos un sistema local de coordenadas en el origen del WP

    #########################
    ###### CARGAR DATOS #####
    #########################

    # Importamos el tren de cargas
    os.chdir(dir_path + "\Parametros_casos_estudio")
    mapdl.cwd(dir_path + "\Parametros_casos_estudio")

    base_load_name = "caso_estudio_palanquilla_"
    current_case_study = base_load_name + case_id + ".csv"

    loads_current_case = load_case_studies_data(current_case_study) # cargamos el tren de cargas correspondiente al caso de estudio actual

    # Importamos las condiciones de contorno de temperaturas
    base_bound_cond_name = "cond_contorno_palanquilla_"
    current_case_study_bound_cond = base_bound_cond_name + case_id + ".csv"

    bound_cond_current_case = load_boundary_conditions(current_case_study_bound_cond) # cargamos las temperaturas de contorno correspondientes al caso actual

    temp_init = bound_cond_current_case.loc[0,"temperatura_inicial"] # temperatura inicial de la barra
    air_temp = bound_cond_current_case.loc[0,"temperatura_aire"] # temperatura del aire circundante

    ######################
    ###### SOLUCION ######
    ######################

    # ----- Primer LoadStep: fijar las condiciones iniciales -----

    mapdl.csys(12) # activamos el sistema de coordenadas que hemos definido anteriormente

    mapdl.nsel('S', 'LOC', 'X', 0.001, -0.001)	    # seleccionamos los nodos en la zona X=0
    mapdl.sf('ALL', 'CONV', conv_coef, temp_init)	# imponemos una convección en los nodos seleccionados
    mapdl.allsel()

    mapdl.nsel('S', 'LOC', 'Y', 0.001, -0.001)	    # seleccionamos los nodos en la zona Y=0
    mapdl.sf('ALL', 'CONV', conv_coef, temp_init)	# imponemos una convección en los nodos seleccionados
    mapdl.allsel()

    mapdl.nsel('S', 'LOC', 'Z', 0.001, -0.001)	    # seleccionamos los nodos en la zona Z=0
    mapdl.sf('ALL', 'CONV', conv_coef, temp_init)	# imponemos una convección en los nodos seleccionados
    mapdl.allsel()

    mapdl.nsel('S', 'LOC', 'X', a+0.001, a-0.001)	# seleccionamos los nodos en la zona X=a
    mapdl.sf('ALL', 'CONV', conv_coef, temp_init)   # imponemos una convección en los nodos seleccionados
    mapdl.allsel()

    mapdl.nsel('S', 'LOC', 'Y', b+0.001, b-0.001)	# seleccionamos los nodos en la zona Y=b
    mapdl.sf('ALL', 'CONV', conv_coef, temp_init)	# imponemos una convección en los nodos seleccionados
    mapdl.allsel()

    mapdl.nsel('S', 'LOC', 'Z', e+0.001, e-0.001)	# seleccionamos los nodos en la zona Z=e
    mapdl.sf('ALL', 'CONV', conv_coef, temp_init)	# imponemos una convección en los nodos seleccionados
    mapdl.allsel()

    mapdl.antype("TRANS")	# transitorio
    mapdl.timint('OFF')	    # se convierte en estacionario
    mapdl.time(0.003)	    # tiempo muy pequeño para resolver el estacionario
    mapdl.autots('ON')      # activamos el ajuste automático de substeps
    mapdl.finish()	        # cierre del resolvedor
    mapdl.run("/SOLU")
    mapdl.solve()

    # ----- Segundo LoadStep: fijar las condiciones del ciclo -----

    mapdl.antype("TRANS")     # análisis transitorio
    mapdl.kbc(0)              # aplicamos cargas RAMPPED
    mapdl.timint("ON")        # transitorio

    mapdl.cswpla(11,0,0,0)  # creamos un sistema local de coordenadas en el origen del WP. Número 11; en cilíndricas (1)
    mapdl.csys(11)          # activamos el sistema de coordenadas previamente definido

    # Comenzamos a recorrer los steps de la simulación
    for k in range(num_steps):

        start_time_step = time.time()

        mapdl.time(time_init_step+loads_current_case.loc[k,"tiempo"]) # tiempo del transitorio en segundos
        mapdl.autots('ON') # activamos el ajuste automático de substeps
        mapdl.outres("ALL", "ALL")

        mapdl.cswpla(11,0,0,0) # creamos un sistema local de coordenadas en el origen del WP. Número 11; en cilíndricas (1)
        mapdl.csys(11)         # activamos el sistema de coordenadas previamente definido

        mapdl.sfdele('ALL', 'ALL')	# eliminamos todas las posibles cargas previemente aplicadas
        mapdl.bfdele('ALL', 'ALL')	# eliminamos todas las posibles cargas previemente aplicadas sobre nodos
        mapdl.sfadele('ALL', 'ALL') # eliminamos todas las posibles cargas previamente aplicadas sobre áreas

        ##### GRUPO DE INDUCTORES 1 (hay tres inductores, del 1-3) #####
        for i in range(3):
            mapdl.nsel('S', 'LOC', 'X', 0 + i * (sep_inductors + influence) + 0.7, influence + i * (sep_inductors + influence) + 0.7)
            mapdl.nsel('R', 'LOC', 'Y', 0.001, b-0.001)
            mapdl.nsel('R', 'LOC', 'Z', 0.001, e-0.001)
            mapdl.bf('ALL', 'HGEN', loads_current_case.loc[k,"IND_1"]*constant_loss)
            mapdl.allsel()

        ##### GRUPO DE INDUCTORES 2 (hay tres inductores, del 4-6) #####
        for i in range(3,6):
            mapdl.nsel('S', 'LOC', 'X', 0 + i * (sep_inductors + influence) + 0.7, influence + i * (sep_inductors + influence) + 0.7)
            mapdl.nsel('R', 'LOC', 'Y', 0.001, b-0.001)
            mapdl.nsel('R', 'LOC', 'Z', 0.001, e-0.001)
            mapdl.bf('ALL', 'HGEN', loads_current_case.loc[k,"IND_2"]*constant_loss)
            mapdl.allsel()

        ##### GRUPO DE INDUCTORES 3 (hay tres inductores, del 7-9) #####
        for i in range(6,9):
            mapdl.nsel('S', 'LOC', 'X', 0 + i * (sep_inductors + influence) + 0.7, influence + i * (sep_inductors + influence) + 0.7)
            mapdl.nsel('R', 'LOC', 'Y', 0.001, b-0.001)
            mapdl.nsel('R', 'LOC', 'Z', 0.001, e-0.001)
            mapdl.bf('ALL', 'HGEN', loads_current_case.loc[k,"IND_3"]*constant_loss)
            mapdl.allsel()
        
        ##### GRUPO DE INDUCTORES 4 (hay tres inductores, del 10-12) #####
        for i in range(9,12):
            mapdl.nsel('S', 'LOC', 'X', 0 + i * (sep_inductors + influence) + 0.7, influence + i * (sep_inductors + influence) + 0.7)
            mapdl.nsel('R', 'LOC', 'Y', 0.001, b-0.001)
            mapdl.nsel('R', 'LOC', 'Z', 0.001, e-0.001)
            mapdl.bf('ALL', 'HGEN', loads_current_case.loc[k,"IND_4"]*constant_loss)
            mapdl.allsel()

        #### DEFINIR LAS CARGAS GENERADAS POR CONVECCIÓN CON EL AIRE ####
        mapdl.nsel('S', 'LOC', 'X', 0.001 - total_advance, -0.001 + total_advance) # seleccionamos los nodos en la zona X=0
        mapdl.sf('ALL', 'CONV', conv_coef, air_temp) # imponemos una convección en los nodos seleccionados
        mapdl.allsel()  

        mapdl.nsel('S', 'LOC', 'Y', 0.001, -0.001) # seleccionamos los nodos en la zona Y=0
        mapdl.sf('ALL', 'CONV', conv_coef, air_temp) # imponemos una convección en los nodos seleccionados
        mapdl.allsel()  

        mapdl.nsel('S', 'LOC', 'Z', 0.001, -0.001)  # seleccionamos los nodos en la zona Z=0
        mapdl.sf('ALL', 'CONV', conv_coef, air_temp) # imponemos una convección en los nodos seleccionados
        mapdl.allsel()  

        mapdl.nsel('S', 'LOC', 'X', -a+0.001-total_advance, -a-0.001+total_advance) # seleccionamos los nodos en la zona X=-a
        mapdl.sf('ALL', 'CONV', conv_coef, air_temp) # imponemos una convección en los nodos seleccionados
        mapdl.allsel()  

        mapdl.nsel('S', 'LOC', 'Y', b+0.001, b-0.001) # seleccionamos los nodos en la zona Y=b
        mapdl.sf('ALL', 'CONV', conv_coef, air_temp) # imponemos una convección en los nodos seleccionados
        mapdl.allsel()  

        mapdl.nsel('S', 'LOC', 'Z', e+0.001, e-0.001) # seleccionamos los nodos en la zona Z=e
        mapdl.sf('ALL', 'CONV', conv_coef, air_temp) # imponemos una convección en los nodos seleccionados
        mapdl.allsel()

        mapdl.solve()

        total_advance += loads_current_case.loc[k,"avance"] # distancia del nuevo sistema de referencia a la posición original

        mapdl.wpoffs(-loads_current_case.loc[k,"avance"]) # avance del WP

        finish_time_step = time.time() 
        print(" -> El step", k+1, " se ha completado correctamente en", time.strftime("%Hh%Mm%Ss", time.gmtime(finish_time_step-start_time_step)), "(", num_steps-k-1, "restantes )")

    mapdl.finish()

    mapdl.wpoffs(+total_advance) # volver el sistema de referencia a 0

    time_init_step += loads_current_case.loc[k,"avance"] # hacer que el siguiente caso continue con el tiempo siguiente

    # Obtenemos resultados relativos a la simulación
    os.chdir(dir_path + "\Resultados")
    mapdl.cwd(dir_path + "\Resultados")

    mapdl.post1() # entramos en el post-proceso

    # Plot de la temperatura en cada punto de la palanquilla
    # mapdl.show("PNG")
    # mapdl.run("PLNSOL,TEMP,,0,")

    mapdl.path('Path_1', 2, 30, 199) # he cambiado 50 por 100
    mapdl.ppath(1, 0, 0, 0.14, 0.07, 11)
    mapdl.ppath(2, 0, -19.5, 0.14, 0.07, 11)  # generar un path a lo largo de la mitad de la cara superior

    mapdl.pdef('', 'TEMP', '', 'NOAVG')
    mapdl.pbc('PATH', '', 0) 

    # ESCRIBIR LOS RESULTADOS EN UN FICHERO DE TEXTO

    # Formateamos la escritura
    nsigfig = 10
    mapdl.header("OFF", "OFF", "OFF", "OFF", "OFF", "OFF")
    mapdl.format("", "E", nsigfig + 9, nsigfig)
    mapdl.page(1e9, "", -1, 240)

    # Sacamos una tabla con los datos del path (posición y temperatura)
    results_to_write = mapdl.prpath('TEMP')  # IMPORTANTE: esta instrucción siempre después del formateo!

    current_case_study_results = "resultados_caso_estudio_palanquilla_" + case_id + ".csv" # fichero en el que se escriben los resultados

    results_in_array = np.genfromtxt(results_to_write.splitlines()[1:]) # obtenemos los resultados numéricos a partir del TXT
    results_in_df = pd.DataFrame(results_in_array, columns=['posicion', 'temperatura']) # guardamos los resultados en un DataFrame
    results_in_df.to_csv(current_case_study_results,sep=',', header = True, index=False) # guardar los resultados en un TXT

    # Representación gráfica
    plt.plot(results_in_df["posicion"], results_in_df["temperatura"], color = "r")
    plt.xlabel('Position (m)')
    plt.ylabel('Temperature (ºC)')
    plt.title("STEEL BILLET " + case_id)
    image_name = "plots-case-studies/plot_caso_estudio_palanquilla_" + case_id + ".png"
    plt.savefig(image_name)
    plt.clf()

    # Tiempos de ejecución del caso de estudio
    finish_time_case_study = time.time()
    print("\n Se ha completado la simulación del caso de estudio", case_id)
    print(" El tiempo empleado han sido", time.strftime("%Hh%Mm%Ss", time.gmtime(finish_time_case_study-start_time_case_study)))
    print("")
# Tiempos de ejecución de la simulación total
finish_time_simul = time.time()
print(75*"=")
print("El tiempo empleado para toda la simulación han sido", time.strftime("%Hh%Mm%Ss", time.gmtime(finish_time_simul-start_time_simul)))
print(75*"=")

              COMIENZA LA SIMULACIÓN - PALANQUILLA 6211285
 -> El step 1  se ha completado correctamente en 00h00m04s ( 199 restantes )
 -> El step 2  se ha completado correctamente en 00h00m01s ( 198 restantes )
 -> El step 3  se ha completado correctamente en 00h00m02s ( 197 restantes )
 -> El step 4  se ha completado correctamente en 00h00m02s ( 196 restantes )
 -> El step 5  se ha completado correctamente en 00h00m01s ( 195 restantes )
 -> El step 6  se ha completado correctamente en 00h00m02s ( 194 restantes )
 -> El step 7  se ha completado correctamente en 00h00m03s ( 193 restantes )
 -> El step 8  se ha completado correctamente en 00h00m01s ( 192 restantes )
 -> El step 9  se ha completado correctamente en 00h00m01s ( 191 restantes )
 -> El step 10  se ha completado correctamente en 00h00m01s ( 190 restantes )
 -> El step 11  se ha completado correctamente en 00h00m01s ( 189 restantes )
 -> El step 12  se ha completado correctamente en 00h00m01s ( 188 restantes )
 -> El step 13

<Figure size 640x480 with 0 Axes>